<a href="https://colab.research.google.com/github/adipurnamk/Belajar-Pengembangan-Machine-Learning/blob/master/Indo_IMDB_LSTM__Multiclass_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Dataset](https://www.kaggle.com/antoniuscs/imdb-synopsis-indonesian-movies)

In [1]:
# melakukan ekstraksi pada file zip
import zipfile,os
local_zip = '/content/88471_203593_bundle_archive.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

os.listdir('/content')

['.config',
 '88471_203593_bundle_archive.zip',
 'imdb_indonesian_movies_2.csv',
 'sample_data']

In [3]:
import pandas as pd
df = pd.read_csv('/content/imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])
df

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama
...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,Romantis
1001,Markonah melarikan diri ke Jakarta karena akan...,Romantis
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",Romantis
1003,Proyek baru ini adalah tentang seorang lelaki ...,Romantis


In [4]:
# One-hot encoding

category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


Agar dapat diproses oleh model, kita perlu mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.

In [5]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [6]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)
 
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [8]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [9]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 25s - loss: 1.6102 - accuracy: 0.1816 - val_loss: 1.6110 - val_accuracy: 0.1592
Epoch 2/30
26/26 - 27s - loss: 1.6082 - accuracy: 0.2388 - val_loss: 1.6109 - val_accuracy: 0.1692
Epoch 3/30
26/26 - 27s - loss: 1.6055 - accuracy: 0.3371 - val_loss: 1.6139 - val_accuracy: 0.2090
Epoch 4/30
26/26 - 27s - loss: 1.4839 - accuracy: 0.4192 - val_loss: 1.5969 - val_accuracy: 0.2040
Epoch 5/30
26/26 - 27s - loss: 1.1131 - accuracy: 0.6045 - val_loss: 1.6948 - val_accuracy: 0.1990
Epoch 6/30
26/26 - 27s - loss: 0.7714 - accuracy: 0.7351 - val_loss: 2.0703 - val_accuracy: 0.2637
Epoch 7/30
26/26 - 27s - loss: 0.3577 - accuracy: 0.8856 - val_loss: 2.7873 - val_accuracy: 0.2687
Epoch 8/30
26/26 - 27s - loss: 0.1490 - accuracy: 0.9664 - val_loss: 3.5280 - val_accuracy: 0.2736
Epoch 9/30
26/26 - 27s - loss: 0.0968 - accuracy: 0.9826 - val_loss: 3.8282 - val_accuracy: 0.2488
Epoch 10/30
26/26 - 27s - loss: 0.0349 - accuracy: 0.9938 - val_loss: 4.3460 - val_accuracy: 0.2587
Epoch 11/